In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('week2.csv')

In [3]:
df.head()

,Player,Team,Opponent,DK Position,DK Salary,DK Projection,DK Value,DK Ownership,DKSlateID,DK Ceiling
0,CeeDee Lamb,DAL,LAC,WR,6400,19.7,4.0,19.6,19184506,29.3
1,Cooper Kupp,LA,IND,WR,6000,18.8,4.0,24.9,19184518,27.8
2,Amari Cooper,DAL,LAC,WR,6800,20.1,3.6,20.1,19184496,30.2
3,Justin Herbert,LAC,DAL,QB,6700,24.3,3.6,15.1,19183997,33.3
4,Dak Prescott,DAL,LAC,QB,6800,24.4,3.4,14.1,19183996,34.8


In [4]:
# Get disinct teams
teams = df['Team'].unique()

In [5]:
# Get position lists
qb_list = df[(df['DK Position'] == 'QB')]
wr_list = df[(df['DK Position'] == 'WR') | (df['DK Position'] == 'TE')]
rb_list = df[(df['DK Position'] == 'RB')]

In [6]:
wr_list

,Player,Team,Opponent,DK Position,DK Salary,DK Projection,DK Value,DK Ownership,DKSlateID,DK Ceiling
0,CeeDee Lamb,DAL,LAC,WR,6400,19.7,4.0,19.6,19184506,29.3
1,Cooper Kupp,LA,IND,WR,6000,18.8,4.0,24.9,19184518,27.8
2,Amari Cooper,DAL,LAC,WR,6800,20.1,3.6,20.1,19184496,30.2
12,Keenan Allen,LAC,DAL,WR,7000,19.4,2.5,17.9,19184492,28.6
17,Chris Godwin,TB,ATL,WR,6600,17.9,1.8,15.9,19184500,28.2
...,...,...,...,...,...,...,...,...,...,...
291,JJ Arcega-Whiteside,PHI,SF,WR,3000,0.3,-8.7,0.0,19184960,2.0
294,N'Keal Harry,NE,NYJ,WR,3000,0.1,-8.9,0.0,19185034,2.0
296,Tyron Johnson,JAX,DEN,WR,3600,0.9,-9.2,0.0,19184620,2.2
297,Willie Snead IV,LV,PIT,WR,3400,0.6,-9.2,0.0,19184650,1.9


In [7]:
stacks = []
runbacks = []
double_stacks = []
double_stacks_runback = []
wr_matchups = []
wr_combos = []
rb_combos = []

for row in qb_list.iterrows():
    qb_name = row[1][0]
    qb_team = row[1][1]
    qb_opponent = row[1][2]
    qb_projection = row[1][5]
    qb_ceiling = row[1][9]
    qb_salary = row[1][4]
    qb_ownership = row[1][7]
    
    #get same team WR
    for wr_row in wr_list[wr_list['Team'] == qb_team].iterrows():
        wr_name = wr_row[1][0]
        wr_projection = wr_row[1][5]
        wr_ceiling = wr_row[1][9]
        wr_salary = wr_row[1][4]
        wr_ownership = wr_row[1][7]
        
        stack_cost = wr_salary + qb_salary
        stack_ownership = qb_ownership + wr_ownership
        stack_projection = qb_projection + wr_projection
        stack_ceiling = qb_ceiling + wr_ceiling
        stack_mean_projection = (stack_projection + stack_ceiling) / 2 
        
        stack_dollar_proj = stack_cost / stack_projection 
        stack_dollar_mean = stack_cost / stack_mean_projection
        stack_dollar_ceiling = stack_cost / stack_ceiling 
        
        #add to stack
        stacks.append([qb_name, wr_name, qb_team, qb_opponent, stack_cost, stack_ownership,
                      stack_projection, stack_mean_projection, stack_ceiling, stack_dollar_proj, stack_dollar_mean, stack_dollar_ceiling])
        
        # get double stack 
        for wr_row in wr_list[(wr_list['Team'] == qb_team) & (wr_list['Player'] != wr_name)].iterrows():
            double_wr_name = wr_row[1][0]
            double_wr_projection = wr_row[1][5]
            double_wr_ceiling = wr_row[1][9]
            double_wr_salary = wr_row[1][4]
            double_wr_ownership = wr_row[1][7]

            double_stack_cost = wr_salary + qb_salary + double_wr_salary
            double_stack_ownership = qb_ownership + wr_ownership + double_wr_ownership
            double_stack_projection = qb_projection + wr_projection + double_wr_projection
            double_stack_ceiling = qb_ceiling + wr_ceiling + double_wr_ceiling
            double_stack_mean_projection = (double_stack_projection + double_stack_ceiling) / 2 

            double_stack_dollar_proj = double_stack_cost / double_stack_projection 
            double_stack_dollar_mean = double_stack_cost / double_stack_mean_projection
            double_stack_dollar_ceiling = double_stack_cost / double_stack_ceiling 
            
            #check for duplicate 
            duplicate_entry = False 
            
            for double_stack_entry in double_stacks: 
                if double_stack_entry[0] == qb_name and double_stack_entry[1] == double_wr_name and double_stack_entry[2] == wr_name:
                    duplicate_entry = True
                    break
            
            if duplicate_entry == False: 
                #add to double stacks
                double_stacks.append([qb_name, wr_name, double_wr_name, qb_team, qb_opponent, double_stack_cost, double_stack_ownership,
                                      double_stack_projection, double_stack_mean_projection, double_stack_ceiling, double_stack_dollar_proj, double_stack_dollar_mean, 
                                      double_stack_dollar_ceiling])
            
        
        #get opp team WR against QB
        for opp_wr_row in wr_list[wr_list['Team'] == qb_opponent].iterrows():
            opp_wr_name = opp_wr_row[1][0]
            opp_wr_projection = opp_wr_row[1][5]
            opp_wr_ceiling = opp_wr_row[1][9]
            opp_wr_salary = opp_wr_row[1][4]
            opp_wr_ownership = opp_wr_row[1][7]
            
            run_back_cost = stack_cost + opp_wr_salary
            run_back_ownership = stack_ownership + opp_wr_ownership
            run_back_projection = stack_projection + opp_wr_projection
            run_back_ceiling = stack_ceiling + opp_wr_ceiling
            run_back_mean = (run_back_projection + run_back_ceiling) / 2
            
            run_back_dollar_proj = run_back_cost / run_back_projection 
            run_back_dollar_mean = run_back_cost / run_back_mean
            run_back_dollar_ceiling = run_back_cost / run_back_ceiling 
            
            # add to stack + run back
            runbacks.append([qb_name, wr_name, opp_wr_name, qb_team, qb_opponent, run_back_cost, 
                             run_back_ownership, run_back_projection, run_back_mean, run_back_ceiling, run_back_dollar_proj, run_back_dollar_mean, run_back_dollar_ceiling])
            
#    *********** End of QB stacks ***********

#Double stack run back
for double_stack in double_stacks:

    ds_qb = double_stack[0]
    ds_wr1 = double_stack[1]
    ds_wr2 = double_stack[2]
    ds_team = double_stack[3]
    ds_opp_team = double_stack[4]
    ds_cost = double_stack[5]
    ds_own = double_stack[6]
    ds_proj = double_stack[7]
    ds_mean_proj = double_stack[8]
    ds_ceiling = double_stack[9]
    ds_dollar_proj = double_stack[10]
    ds_dollar_celing = double_stack[11]

    for ds_opp_wr_row in wr_list[wr_list['Team'] == ds_opp_team].iterrows():
        ds_opp_wr_name = ds_opp_wr_row[1][0]
        ds_opp_wr_projection = ds_opp_wr_row[1][5]
        ds_opp_wr_ceiling = ds_opp_wr_row[1][9]
        ds_opp_wr_salary = ds_opp_wr_row[1][4]
        ds_opp_wr_ownership = ds_opp_wr_row[1][7]

        ds_run_back_cost = ds_cost + ds_opp_wr_salary
        ds_run_back_ownership = ds_own + ds_opp_wr_ownership
        ds_run_back_projection = ds_proj + ds_opp_wr_projection
        ds_run_back_ceiling = ds_ceiling + ds_opp_wr_ceiling
        ds_run_back_mean = (ds_run_back_projection + ds_run_back_ceiling) / 2

        ds_run_back_dollar_proj = ds_run_back_cost / ds_run_back_projection 
        ds_run_back_dollar_mean = ds_run_back_cost / ds_run_back_mean
        ds_run_back_dollar_ceiling = ds_run_back_cost / ds_run_back_ceiling 
        
        double_stacks_runback.append([ds_qb, ds_wr1, ds_wr2, ds_opp_wr_name, ds_team, ds_opp_team, ds_run_back_cost, 
                                     ds_run_back_ownership, ds_run_back_projection, ds_run_back_mean, 
                                     ds_run_back_ceiling, ds_run_back_dollar_proj, ds_run_back_dollar_mean, 
                                     ds_run_back_dollar_ceiling])
    
# get WR/TE match ups - 2 WR/TE on opposing teams
for wr1 in wr_list.iterrows(): 
    wr1_name = wr1[1][0]
    wr1_team = wr1[1][1]
    wr1_opponent = wr1[1][2]
    wr1_projection = wr1[1][5]
    wr1_ceiling = wr1[1][9]
    wr1_salary = wr1[1][4]
    wr1_ownership = wr1[1][7]

    for wr2 in wr_list[wr_list['Team'] == wr1_opponent].iterrows():
            wr2_name = wr2[1][0]
            wr2_projection = wr2[1][5]
            wr2_ceiling = wr2[1][9]
            wr2_salary = wr2[1][4]
            wr2_ownership = wr2[1][7]

            wr_matchup_cost = wr1_salary + wr2_salary
            wr_matchup_ownership = wr1_ownership + wr2_ownership
            wr_matchup_projection = wr1_projection + wr2_projection
            wr_matchup_ceiling = wr1_ceiling + wr2_ceiling
            wr_matchup_mean_projection = (wr_matchup_projection + wr_matchup_ceiling) / 2 

            wr_matchup_dollar_proj = wr_matchup_cost / wr_matchup_projection 
            wr_matchup_dollar_mean = wr_matchup_cost / wr_matchup_mean_projection
            wr_matchup_dollar_ceiling = wr_matchup_cost / wr_matchup_ceiling 

            #check for duplicate 
            duplicate_entry = False 

            for wr_matchup in wr_matchups: 
                if wr_matchup[0] == wr2_name and wr_matchup[1] == wr1_name:
                    duplicate_entry = True
                    break


            if duplicate_entry == False: 
                #add to double stacks
                wr_matchups.append([wr1_name, wr2_name, wr1_team, wr1_opponent, wr_matchup_cost, wr_matchup_ownership,
                                      wr_matchup_projection, wr_matchup_mean_projection, wr_matchup_ceiling, wr_matchup_dollar_proj, wr_matchup_dollar_mean, 
                                      wr_matchup_dollar_ceiling])
    
    
# get WR combos (2 teams not just opponents)

# get WR/TE combos - 2 WR/TE on any teams
for wr1 in wr_list.iterrows(): 
    wr1_name = wr1[1][0]
    wr1_team = wr1[1][1]
    wr1_projection = wr1[1][5]
    wr1_ceiling = wr1[1][9]
    wr1_salary = wr1[1][4]
    wr1_ownership = wr1[1][7]

    for wr2 in wr_list[wr_list['Player'] != wr1_name].iterrows():
            wr2_name = wr2[1][0]
            wr2_team = wr2[1][1]
            wr2_projection = wr2[1][5]
            wr2_ceiling = wr2[1][9]
            wr2_salary = wr2[1][4]
            wr2_ownership = wr2[1][7]

            wr_combo_cost = wr1_salary + wr2_salary
            wr_combo_ownership = wr1_ownership + wr2_ownership
            wr_combo_projection = wr1_projection + wr2_projection
            wr_combo_ceiling = wr1_ceiling + wr2_ceiling
            wr_combo_mean_projection = (wr_combo_projection + wr_combo_ceiling) / 2 

            wr_combo_dollar_proj = wr_combo_cost / wr_combo_projection 
            wr_combo_dollar_mean = wr_combo_cost / wr_combo_mean_projection
            wr_combo_dollar_ceiling = wr_combo_cost / wr_combo_ceiling 

            #check for duplicate 
            duplicate_entry = False 

            for wr_combo in wr_combos: 
                if wr_combo[0] == wr2_name and wr_combo[1] == wr1_name:
                    duplicate_entry = True
                    break


            if duplicate_entry == False: 
                #add to double stacks
                wr_combos.append([wr1_name, wr2_name, wr1_team, wr2_team, wr_combo_cost, wr_combo_ownership,
                                      wr_combo_projection, wr_combo_mean_projection, wr_combo_ceiling, wr_combo_dollar_proj, wr_combo_dollar_mean, 
                                      wr_combo_dollar_ceiling])



# get RB combos (2 teams not just opponents)
for rb1 in rb_list.iterrows(): 
    rb1_name = rb1[1][0]
    rb1_team = rb1[1][1]
    rb1_projection = rb1[1][5]
    rb1_ceiling = rb1[1][9]
    rb1_salary = rb1[1][4]
    rb1_ownership = rb1[1][7]

    for rb2 in rb_list[rb_list['Player'] != rb1_name].iterrows():
            rb2_name = rb2[1][0]
            rb2_team = rb2[1][1]
            rb2_projection = rb2[1][5]
            rb2_ceiling = rb2[1][9]
            rb2_salary = rb2[1][4]
            rb2_ownership = rb2[1][7]

            rb_combo_cost = rb1_salary + rb2_salary
            rb_combo_ownership = rb1_ownership + rb2_ownership
            rb_combo_projection = rb1_projection + rb2_projection
            rb_combo_ceiling = rb1_ceiling + rb2_ceiling
            rb_combo_mean_projection = (rb_combo_projection + rb_combo_ceiling) / 2 

            rb_combo_dollar_proj = rb_combo_cost / rb_combo_projection 
            rb_combo_dollar_mean = rb_combo_cost / rb_combo_mean_projection
            rb_combo_dollar_ceiling = rb_combo_cost / rb_combo_ceiling 

            #check for duplicate 
            duplicate_entry = False 

            for rb_combo in rb_combos: 
                if rb_combo[0] == rb2_name and rb_combo[1] == rb1_name:
                    duplicate_entry = True
                    break


            if duplicate_entry == False: 
                #add to double stacks
                rb_combos.append([rb1_name, rb2_name, rb1_team, rb2_team, rb_combo_cost, rb_combo_ownership,
                                    rb_combo_projection, rb_combo_mean_projection, rb_combo_ceiling, rb_combo_dollar_proj, rb_combo_dollar_mean, 
                                      rb_combo_dollar_ceiling])
    

In [8]:
# Stacks
stacks_df = pd.DataFrame(stacks)
stacks_df.columns = ['QB', 'Team_WR', 'Team', 'Opp', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                    'Proj_Pts_Cost', 'Mean_Pts_Cost', 'Ceiling_Pts_Cost']
stacks_df.sort_values('Ceiling_Pts', ascending=False).head(50)

,QB,Team_WR,Team,Opp,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
9,Dak Prescott,Amari Cooper,DAL,LAC,13600,34.2,44.5,54.75,65.0,305.617978,248.401826,209.230769
52,Russell Wilson,Tyler Lockett,SEA,TEN,14700,11.8,43.7,54.20,64.7,336.384439,271.217712,227.202473
24,Josh Allen,Stefon Diggs,BUF,MIA,14900,25.7,43.6,54.05,64.5,341.743119,275.670675,231.007752
8,Dak Prescott,CeeDee Lamb,DAL,LAC,13200,33.7,44.1,54.10,64.1,299.319728,243.992606,205.928237
53,Russell Wilson,DK Metcalf,SEA,TEN,15100,10.3,43.7,53.85,64.0,345.537757,280.408542,235.937500
15,Tom Brady,Chris Godwin,TB,ATL,13500,26.7,42.2,52.60,63.0,319.905213,256.653992,214.285714
92,Kyler Murray,DeAndre Hopkins,ARI,MIN,16200,12.6,43.4,53.00,62.6,373.271889,305.660377,258.785942
0,Justin Herbert,Keenan Allen,LAC,DAL,13700,33.0,43.7,52.80,61.9,313.501144,259.469697,221.324717
17,Tom Brady,Mike Evans,TB,ATL,13000,17.1,39.3,49.05,58.8,330.788804,265.035678,221.088435
16,Tom Brady,Antonio Brown,TB,ATL,12900,15.8,39.3,48.85,58.4,328.244275,264.073695,220.890411


In [9]:
# Double Stacks
double_stacks_df = pd.DataFrame(double_stacks)
double_stacks_df.columns = ['QB', 'Team_WR', 'Team_WR_2', 'Team', 'Opp', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

double_stacks_df.sort_values('Ceiling_Pts', ascending=False).head(50)

,QB,Team_WR,Team_WR_2,Team,Opp,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
28,Dak Prescott,CeeDee Lamb,Amari Cooper,DAL,LAC,20000,53.8,64.2,79.25,94.3,311.526480,252.365931,212.089077
168,Russell Wilson,Tyler Lockett,DK Metcalf,SEA,TEN,22300,16.4,62.5,77.95,93.4,356.800000,286.080821,238.758030
50,Tom Brady,Chris Godwin,Mike Evans,TB,ATL,19600,33.0,57.2,72.10,87.0,342.657343,271.844660,225.287356
49,Tom Brady,Chris Godwin,Antonio Brown,TB,ATL,19500,31.7,57.2,71.90,86.6,340.909091,271.210014,225.173210
1,Justin Herbert,Keenan Allen,Mike Williams,LAC,DAL,19800,35.9,59.1,72.65,86.2,335.025381,272.539573,229.698376
77,Josh Allen,Cole Beasley,Stefon Diggs,BUF,MIA,19500,38.5,56.6,70.40,84.2,344.522968,276.988636,231.591449
56,Tom Brady,Antonio Brown,Mike Evans,TB,ATL,19000,22.1,54.3,68.35,82.4,349.907919,277.980980,230.582524
83,Josh Allen,Stefon Diggs,Emmanuel Sanders,BUF,MIA,18900,35.0,54.9,68.25,81.6,344.262295,276.923077,231.617647
51,Tom Brady,Chris Godwin,Rob Gronkowski,TB,ATL,18200,28.4,52.8,66.75,80.7,344.696970,272.659176,225.526642
0,Justin Herbert,Keenan Allen,Jared Cook,LAC,DAL,17600,46.6,55.3,67.95,80.6,318.264014,259.013981,218.362283


In [10]:
# Double Stacks with runback
double_stacks_runback_df = pd.DataFrame(double_stacks_runback)
double_stacks_runback_df.columns = ['QB', 'Team_WR', 'Team_WR_2', 'Opp_WR', 'Team', 'Opp', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

double_stacks_runback_df.sort_values('Ceiling_Pts', ascending=False).head(50)

,QB,Team_WR,Team_WR_2,Opp_WR,Team,Opp,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
196,Dak Prescott,CeeDee Lamb,Amari Cooper,Keenan Allen,DAL,LAC,27000,71.7,83.6,103.25,122.9,322.966507,261.501211,219.690806
1288,Russell Wilson,Tyler Lockett,DK Metcalf,A.J. Brown,SEA,TEN,29200,28.3,80.3,100.25,120.2,363.636364,291.271820,242.928453
198,Dak Prescott,CeeDee Lamb,Amari Cooper,Mike Williams,DAL,LAC,26100,56.7,79.6,99.10,118.6,327.889447,263.370333,220.067454
8,Justin Herbert,Keenan Allen,Mike Williams,Amari Cooper,LAC,DAL,26600,56.0,79.2,97.80,116.4,335.858586,271.983640,228.522337
1289,Russell Wilson,Tyler Lockett,DK Metcalf,Julio Jones,SEA,TEN,28600,19.6,77.8,97.00,116.2,367.609254,294.845361,246.127367
7,Justin Herbert,Keenan Allen,Mike Williams,CeeDee Lamb,LAC,DAL,26200,55.5,78.8,97.15,115.5,332.487310,269.686052,226.839827
372,Tom Brady,Chris Godwin,Mike Evans,Calvin Ridley,TB,ATL,27100,42.3,75.9,94.60,113.3,357.048748,286.469345,239.187996
197,Dak Prescott,CeeDee Lamb,Amari Cooper,Jared Cook,DAL,LAC,23900,67.4,75.8,94.40,113.0,315.303430,253.177966,211.504425
364,Tom Brady,Chris Godwin,Antonio Brown,Calvin Ridley,TB,ATL,27000,41.0,75.9,94.40,112.9,355.731225,286.016949,239.149690
1,Justin Herbert,Keenan Allen,Jared Cook,Amari Cooper,LAC,DAL,24400,66.7,75.4,93.10,110.8,323.607427,262.083781,220.216606


In [11]:
# Stack with Runback
runbacks_df = pd.DataFrame(runbacks)
runbacks_df.columns = ['QB', 'Team_WR', 'Opp_WR', 'Team', 'Opp', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

runbacks_df.sort_values('Ceiling_Pts', ascending=False).head(50)

,QB,Team_WR,Opp_WR,Team,Opp,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
64,Dak Prescott,Amari Cooper,Keenan Allen,DAL,LAC,20600,52.1,63.9,78.75,93.6,322.378717,261.587302,220.085470
56,Dak Prescott,CeeDee Lamb,Keenan Allen,DAL,LAC,20200,51.6,63.5,78.10,92.7,318.110236,258.642766,217.907228
1,Justin Herbert,Keenan Allen,Amari Cooper,LAC,DAL,20500,53.1,63.8,77.95,92.1,321.316614,262.989096,222.584148
400,Russell Wilson,Tyler Lockett,A.J. Brown,SEA,TEN,21600,23.7,61.5,76.50,91.5,351.219512,282.352941,236.065574
0,Justin Herbert,Keenan Allen,CeeDee Lamb,LAC,DAL,20100,52.6,63.4,77.30,91.2,317.034700,260.025873,220.394737
406,Russell Wilson,DK Metcalf,A.J. Brown,SEA,TEN,22000,22.2,61.5,76.15,90.8,357.723577,288.903480,242.290749
66,Dak Prescott,Amari Cooper,Mike Williams,DAL,LAC,19700,37.1,59.9,74.60,89.3,328.881469,264.075067,220.604703
112,Tom Brady,Chris Godwin,Calvin Ridley,TB,ATL,21000,36.0,60.9,75.10,89.3,344.827586,279.627164,235.162374
58,Dak Prescott,CeeDee Lamb,Mike Williams,DAL,LAC,19300,36.6,59.5,73.95,88.4,324.369748,260.987153,218.325792
15,Justin Herbert,Mike Williams,Amari Cooper,LAC,DAL,19600,38.1,59.8,73.80,87.8,327.759197,265.582656,223.234624


In [19]:
# WR/TE Match Ups (Same Game - Opposing Teams)
wr_matchups_df = pd.DataFrame(wr_matchups)
wr_matchups_df.columns = ['WR1', 'WR2', 'WR1 Team', 'WR2 Team', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

wr_matchups_df[wr_matchups_df['Total_Salary'] < 10900].sort_values('Ceiling_Pts', ascending=False).head(50)

,WR1,WR2,WR1 Team,WR2 Team,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
17,Amari Cooper,Jared Cook,DAL,LAC,10700,33.7,31.7,40.30,48.9,337.539432,265.508685,218.813906
1,CeeDee Lamb,Jared Cook,DAL,LAC,10300,33.2,31.3,39.65,48.0,329.073482,259.773014,214.583333
24,Keenan Allen,Cedrick Wilson,LAC,DAL,10100,21.4,28.1,35.10,42.1,359.430605,287.749288,239.904988
19,Amari Cooper,Jalen Guyton,DAL,LAC,9900,20.6,26.7,34.35,42.0,370.786517,288.209607,235.714286
8,Cooper Kupp,Michael Pittman Jr.,LA,IND,10300,27.4,28.2,34.65,41.1,365.248227,297.258297,250.608273
3,CeeDee Lamb,Jalen Guyton,DAL,LAC,9500,20.1,26.3,33.70,41.1,361.216730,281.899110,231.143552
46,Tyler Lockett,Anthony Firkser,SEA,TEN,10600,6.9,26.5,33.65,40.8,400.000000,315.007429,259.803922
25,Keenan Allen,Dalton Schultz,LAC,DAL,10300,18.9,26.3,33.00,39.7,391.634981,312.121212,259.445844
80,A.J. Brown,Gerald Everett,TEN,SEA,10500,15.3,25.4,32.50,39.6,413.385827,323.076923,265.151515
47,Tyler Lockett,Chester Rogers,SEA,TEN,10400,6.6,25.5,32.55,39.6,407.843137,319.508449,262.626263


In [23]:
# WR/TE Combos (2 Players Any Team)
wr_combos_df = pd.DataFrame(wr_combos)
wr_combos_df.columns = ['WR1', 'WR2', 'WR1 Team', 'WR2 Team', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

wr_combos_df[wr_combos_df['Total_Salary'] < 9900].sort_values('Ceiling_Pts', ascending=False).head(50)

,WR1,WR2,WR1 Team,WR2 Team,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
418,Amari Cooper,Donovan Peoples-Jones,DAL,CLE,9800,21.2,27.9,36.15,44.4,351.254480,271.092669,220.720721
49,CeeDee Lamb,Donovan Peoples-Jones,DAL,CLE,9400,20.7,27.5,35.50,43.5,341.818182,264.788732,216.091954
57,CeeDee Lamb,Anthony Schwartz,DAL,CLE,9700,20.1,27.6,35.50,43.4,351.449275,273.239437,223.502304
406,Amari Cooper,Adam Trautman,DAL,NO,9800,32.7,28.7,35.90,43.1,341.463415,272.980501,227.378190
223,Cooper Kupp,A.J. Green,LA,ARI,9700,28.4,28.6,35.75,42.9,339.160839,271.328671,226.107226
40,CeeDee Lamb,Cedrick Wilson,DAL,DAL,9500,23.1,28.4,35.60,42.8,334.507042,266.853933,221.962617
431,Amari Cooper,Will Dissly,DAL,SEA,9800,21.3,27.1,34.90,42.7,361.623616,280.802292,229.508197
783,Chris Godwin,Donovan Peoples-Jones,TB,CLE,9600,17.0,25.7,34.05,42.4,373.540856,281.938326,226.415094
37,CeeDee Lamb,Adam Trautman,DAL,NO,9400,32.2,28.3,35.25,42.2,332.155477,266.666667,222.748815
219,Cooper Kupp,Cole Kmet,LA,CHI,9700,29.4,28.7,35.40,42.1,337.979094,274.011299,230.403800


In [24]:
# RB Combos (2 Players)
rb_matchups_df = pd.DataFrame(rb_combos)
rb_matchups_df.columns = ['RB1', 'RB2', 'RB1 Team', 'RB2 Team', 'Total_Salary',
                   'Total_Ownership', 'Projected_Pts', 'Mean_Pts', 'Ceiling_Pts',
                      'Proj_Pts_Cost','Mean_Pts_Cost',  'Ceiling_Pts_Cost']

rb_matchups_df[rb_matchups_df['Total_Salary'] < 13500].sort_values('Ceiling_Pts', ascending=False).head(50)

,RB1,RB2,RB1 Team,RB2 Team,Total_Salary,Total_Ownership,Projected_Pts,Mean_Pts,Ceiling_Pts,Proj_Pts_Cost,Mean_Pts_Cost,Ceiling_Pts_Cost
390,Damien Harris,Nick Chubb,NE,CLE,13200,29.4,35.8,44.25,52.7,368.715084,298.305085,250.474383
8,Chris Carson,Austin Ekeler,SEA,LAC,13400,35.8,36.6,44.55,52.5,366.120219,300.785634,255.238095
1,Chris Carson,Najee Harris,SEA,PIT,12400,53.1,36.7,44.35,52.0,337.874659,279.594138,238.461538
164,Najee Harris,Joe Mixon,PIT,CIN,13300,37.0,36.6,43.60,50.6,363.387978,305.045872,262.845850
90,Alvin Kamara,Javonte Williams,NO,DEN,13200,19.4,35.7,43.10,50.5,369.747899,306.264501,261.386139
7,Chris Carson,Joe Mixon,SEA,CIN,13100,38.3,36.3,43.35,50.4,360.881543,302.191465,259.920635
393,Damien Harris,Austin Ekeler,NE,LAC,12700,26.6,34.1,41.75,49.4,372.434018,304.191617,257.085020
540,Darrell Henderson Jr.,Austin Ekeler,LA,LAC,13000,21.3,34.2,41.65,49.1,380.116959,312.124850,264.765784
161,Najee Harris,Damien Harris,PIT,NE,11700,43.9,34.2,41.55,48.9,342.105263,281.588448,239.263804
166,Najee Harris,Ezekiel Elliott,PIT,DAL,12500,44.7,34.7,41.80,48.9,360.230548,299.043062,255.623722
